# Hola, mundo en LangChain

## Instalar librerías principales y configuración de API Key de OpenAI

In [ ]:
!pip install langchain langchain-community pypdf openai chromadb tiktoken

In [9]:
import langchain

In [5]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Carga de documents

In [12]:
import requests
from langchain_community.document_loaders import PyPDFLoader

urls = [
    'https://arxiv.org/pdf/2602.00053',
    'https://arxiv.org/pdf/2602.00266',
    'https://arxiv.org/pdf/2602.00327',
    'https://arxiv.org/pdf/2602.00449',
    'https://arxiv.org/pdf/2602.00561'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper1.pdf
Descargado paper2.pdf
Descargado paper3.pdf
Descargado paper4.pdf
Descargado paper5.pdf
Contenido de ml_papers:



In [ ]:
type(ml_papers), len(ml_papers), ml_papers[3] #clase DOCUMENT -> es la clave para langchain

(list,
 117,
 Document(metadata={'producer': 'pikepdf 8.15.1', 'creator': 'arXiv GenPDF (tex2pdf:57610bf)', 'creationdate': '', 'author': 'Ratul Ali', 'doi': 'https://doi.org/10.48550/arXiv.2602.00053', 'license': 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.28 (TeX Live 2025) kpathsea version 6.4.1', 'title': 'Scalable and Secure AI Inference in Healthcare: A Comparative Benchmarking of FastAPI and Triton Inference Server on Kubernetes', 'trapped': '/False', 'arxivid': 'https://arxiv.org/abs/2602.00053v1', 'source': 'paper1.pdf', 'total_pages': 4, 'page': 3, 'page_label': '4'}, page_content='drift detection. Integrating statistical monitoring to detect\nwhen clinical data distributions shift is a critical next step for\npatient safety.\nX. CONCLUSION\nThis benchmarking study provides empirical evidence guid-\ning the selection of inference frameworks for healthcare AI.\nWe conclude that:\n•FastAPIis best suited

## Split de documents

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, #cada texto tendrá 1500 caracteres
    chunk_overlap=200, #200 caracteres de overlapping (para la conexión entre documentos)
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [18]:
len(documents), documents[10]

(348,
 Document(metadata={'producer': 'pikepdf 8.15.1', 'creator': 'arXiv GenPDF (tex2pdf:57610bf)', 'creationdate': '', 'author': 'Ratul Ali', 'doi': 'https://doi.org/10.48550/arXiv.2602.00053', 'license': 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.28 (TeX Live 2025) kpathsea version 6.4.1', 'title': 'Scalable and Secure AI Inference in Healthcare: A Comparative Benchmarking of FastAPI and Triton Inference Server on Kubernetes', 'trapped': '/False', 'arxivid': 'https://arxiv.org/abs/2602.00053v1', 'source': 'paper1.pdf', 'total_pages': 4, 'page': 2, 'page_label': '3'}, page_content='autoscalingusing NVIDIA DCGM metrics to scale based on\nGPU duty cycles.\n2. Edge Deployments:The current architecture is designed\nfor cloud/data center clusters. Adapting this stack for\nlightweightedge devices(e.g., in-hospital servers) remains a\nchallenge.\n3. Model Drift:The current pipeline lacks automated'))

## Embeddings e ingesta a base de datos vectorial

In [22]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # or -large
)

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Modelos de chat y cadenas para consulta de información

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [10]:
query = "qué es fingpt?"
qa_chain.run(query)

'FinGPT es un modelo de lenguaje financiero de código abierto desarrollado por la comunidad AI4Finance. Está diseñado para aplicaciones en el campo de las finanzas y utiliza técnicas de procesamiento de lenguaje natural para analizar y comprender datos financieros. FinGPT adopta un enfoque centrado en los datos y utiliza métodos rigurosos de limpieza y preprocesamiento para garantizar la calidad de los datos. También ofrece un marco de trabajo de extremo a extremo con cuatro capas, que abarca desde la obtención de datos hasta la implementación de aplicaciones prácticas en finanzas. El objetivo de FinGPT es estimular la innovación, democratizar los modelos de lenguaje financiero y desbloquear nuevas oportunidades en el campo de las finanzas abiertas.'

In [11]:
query = "qué hace complicado entrenar un modelo como el fingpt?"
qa_chain.run(query)

'Entrenar un modelo como FinGPT puede ser complicado por varias razones. En primer lugar, los modelos de lenguaje grandes requieren una gran cantidad de recursos computacionales, como unidades de procesamiento gráfico (GPU) y tiempo de entrenamiento. En el caso de BloombergGPT, se utilizaron aproximadamente 1.3 millones de horas de GPU para el entrenamiento, lo que se traduce en un costo significativo.\n\nAdemás, la adquisición de datos financieros de alta calidad y relevantes puede ser un desafío. Obtener datos financieros históricos o especializados de diferentes fuentes, como plataformas web, API, documentos PDF e imágenes, puede ser complejo debido a los diferentes formatos y tipos de datos.\n\nOtro desafío es la naturaleza dinámica del dominio financiero. El mundo financiero está en constante cambio, y los modelos de lenguaje deben actualizarse regularmente para mantener su precisión y relevancia. Esto requiere una capacidad de adaptación rápida y eficiente del modelo, lo cual pue

In [12]:
query = "qué es fast segment?"
qa_chain.run(query)

'Fast Segment es un método alternativo propuesto en el artículo "Fast Segment Anything" para acelerar el modelo Segment Anything (SAM) en tareas de visión por computadora. SAM es un modelo que puede segmentar cualquier objeto en una imagen y ha demostrado resultados prometedores en diversas tareas. Sin embargo, SAM tiene altos costos computacionales debido a su arquitectura Transformer en entradas de alta resolución. Fast Segment propone una forma más rápida de lograr resultados comparables al entrenar un método existente de segmentación de instancias utilizando solo una fracción del conjunto de datos utilizado por SAM. Con Fast Segment, se logra una velocidad de ejecución 50 veces más rápida que SAM sin comprometer significativamente el rendimiento.'

In [13]:
query = "cuál es la diferencia entre fast sam y mobile sam?"
qa_chain.run(query)

'La diferencia entre FastSAM y MobileSAM se puede resumir en dos aspectos principales: tamaño y velocidad.\n\nEn cuanto al tamaño, MobileSAM es significativamente más pequeño que FastSAM. MobileSAM tiene menos de 10 millones de parámetros, mientras que FastSAM tiene 68 millones de parámetros.\n\nEn cuanto a la velocidad, MobileSAM es más rápido que FastSAM. En una GPU individual, MobileSAM tarda solo 10 ms en procesar una imagen, mientras que FastSAM tarda 40 ms. Esto significa que MobileSAM es 4 veces más rápido que FastSAM.\n\nAdemás de estas diferencias, también se menciona que MobileSAM tiene un rendimiento superior en términos de mIoU (Índice de Jaccard medio) en comparación con FastSAM. Esto sugiere que la predicción de máscaras de MobileSAM es más similar a la del SAM original que la de FastSAM.\n\nEn resumen, MobileSAM es más pequeño, más rápido y tiene un rendimiento superior en comparación con FastSAM.'